SHAP

In [ ]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Optional: Import TabPFN if needed
try:
    from tabpfn import TabPFNClassifier
except ImportError:
    TabPFNClassifier = None  # Avoid breaking if not installed

In [ ]:
def getFeatureNamesAndAliasMap():

  # Original feature names
  feature_names = ['FreqGoProvider', 'Deaf', 'MedConditions_Diabetes', 'MedConditions_HighBP', 'MedConditions_LungDisease', 'MedConditions_Depression', 'AverageSleepNight', 'AverageTimeSitting', 'EverHadCancer', 'Age', 'BirthGender', 'BMI', 'PHQ4', 'WeeklyMinutesModerateExercise', 'AvgDrinksPerWeek', 'GeneralHealth_Excellent', 'GeneralHealth_VeryGood', 'GeneralHealth_Good', 'GeneralHealth_Fair', 'GeneralHealth_Poor', 'smokeStat_Current', 'smokeStat_Former', 'smokeStat_Never', 'eCigUse_Current', 'eCigUse_Former', 'eCigUse_Never']
  # Define alias map if not provided ===
  feature_alias_map = {
    'FreqGoProvider': 'Number of Doctor Visits',
    'Deaf': 'Hearing Impaired',
    'MedConditions_Diabetes': 'Diagnosed Diabetes',
    'MedConditions_HighBP': 'Diagnosed High Blood Pressure',
    'MedConditions_HeartCondition': 'Diagnosed Heart Condition',
    'MedConditions_LungDisease': 'Diagnosed Lung Disease',
    'MedConditions_Depression': 'Diagnosed Depression',
    'AverageSleepNight': 'Average Sleep (hrs/night)',
    'AverageTimeSitting': 'Avg Daily Sitting Time',
    'EverHadCancer': 'Ever Diagnosed with Cancer',
    'Age': 'Age (Yrs)',
    'BirthGender': 'Biological Sex',
    'BMI': 'Body Mass Index',
    'PHQ4': 'PHQ-4 (Anxiety/Depression Score)',
    'WeeklyMinutesModerateExercise': 'Moderate Exercise Minutes/Week',
    'AvgDrinksPerWeek': 'Average Drinks/Week',
    'GeneralHealth_Excellent': 'Excellent General Health',
    'GeneralHealth_VeryGood': 'Very Good General Health',
    'GeneralHealth_Good': 'Good General Health',
    'GeneralHealth_Fair': 'Fair General Health',
    'GeneralHealth_Poor': 'Poor General Health',
    'smokeStat_Current': 'Current Smoker',
    'smokeStat_Former': 'Former Smoker',
    'smokeStat_Never': 'Never Smoker',
    'eCigUse_Current': 'Currently Uses E-Cigarettes',
    'eCigUse_Former': 'Former E-Cigarette User',
    'eCigUse_Never': 'Never Used E-Cigarettes'
  }
  return feature_names, feature_alias_map


In [ ]:
def explain_model_shap(
    model,
    X_test,
    return_importance=False,
    feature_alias_map=None,
    feature_names=None
):
    """
    Explains a model's predictions using SHAP.
    Supports: Logistic Regression, XGBoost (Booster, XGBClassifier), Random Forest,
              TabNet, TabPFN, SAINT, or other models with a .predict method.

    Requires: import numpy as np, import pandas as pd, import shap, import xgboost as xgb
              from sklearn.linear_model import LogisticRegression
              from sklearn.ensemble import RandomForestClassifier
    """

    # === Step 1: Validate and prepare input ===
    if not isinstance(X_test, pd.DataFrame):
        raise ValueError("X_test must be a pandas DataFrame with column names")

    X_test_named = X_test.reset_index(drop=True).copy()

    # If the model stores training feature names, enforce exact order & presence
    if hasattr(model, "feature_names_in_"):
        expect = list(model.feature_names_in_)
        missing = [c for c in expect if c not in X_test_named.columns]
        if missing:
            raise ValueError(f"Missing expected training columns: {missing}")
        X_test_named = X_test_named.loc[:, expect]
        ordered_feature_names = expect
    else:
        ordered_feature_names = X_test_named.columns.tolist()

    # === Step 2: Build display names (aliases) WITHOUT renaming data ===
    if feature_alias_map:
        display_names = [feature_alias_map.get(c, c) for c in ordered_feature_names]
    else:
        display_names = ordered_feature_names

    # === Step 3: SHAP explainer logic ===
    try:
        if isinstance(model, xgb.Booster):
            dtest = xgb.DMatrix(X_test_named, feature_names=ordered_feature_names)
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(dtest)

        elif isinstance(model, (xgb.XGBClassifier, RandomForestClassifier)):
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_test_named)

        elif isinstance(model, LogisticRegression):
            # Use NUMPY with LinearExplainer to avoid pandas rename/index issues
            X_np = X_test_named.to_numpy()
            explainer = shap.LinearExplainer(model, X_np)
            shap_values = explainer.shap_values(X_np)

        elif ("TabPFNClassifier" in globals()) and isinstance(model, globals()["TabPFNClassifier"]):
            predict_fn = lambda x: model.predict(x.values if isinstance(x, pd.DataFrame) else x)
            explainer = shap.Explainer(predict_fn, X_test_named)
            shap_values = explainer(X_test_named)

        else:
            predict_fn = lambda x: model.predict(x.values if isinstance(x, pd.DataFrame) else x)
            explainer = shap.Explainer(predict_fn, X_test_named)
            shap_values = explainer(X_test_named)

    except Exception as e:
        raise RuntimeError(f"SHAP explanation failed for model type {type(model)}: {e}")

    # === Step 4: Compute SHAP feature importance (optional) ===
    importance_df = None
    if return_importance:
        try:
            # shap_values can be:
            # - array (n, p)
            # - list of arrays (K classes, each (n, p))
            if isinstance(shap_values, list):
                arr = np.stack([np.asarray(sv) for sv in shap_values], axis=0)  # (K, n, p)
                mean_shap = arr.mean(axis=(0, 1))
                mean_abs_shap = np.abs(arr).mean(axis=(0, 1))
            else:
                arr = shap_values.values if hasattr(shap_values, "values") else np.asarray(shap_values)
                mean_shap = arr.mean(axis=0)
                mean_abs_shap = np.abs(arr).mean(axis=0)

            importance_df = (
                pd.DataFrame({
                    'feature': display_names,
                    'mean_shap': mean_shap,
                    'mean_abs_shap': mean_abs_shap
                })
                .sort_values(by='mean_abs_shap', ascending=False)
                .reset_index(drop=True)
            )

            print("\n🔹 Feature Importance Table (mean SHAP and |SHAP|):")
            print(importance_df)
        except Exception as e:
            print(f"⚠️ Unable to compute importance_df: {e}")

    # === Step 5: Return output (keep original contract: 2 or 3 values) ===
    return (shap_values, explainer, importance_df) if return_importance else (shap_values, explainer)


In [ ]:
def plot_shap(
    shap_values,
    explainer,
    importance_df,              # accepted for backward-compat; not required for plotting
    X_test,
    show_summary=True,
    show_bar=True,
    show_waterfall=True,
    feature_alias_map=None,
    feature_names=None,         # optional explicit order (original training names)
    class_index=None,           # which class to plot if multiclass
    clean_labels=False          # set True to auto-clean alias labels (remove () and slashes)
):
    """
    SHAP plotting helper that:
      - NEVER renames X_test columns (keeps original names for SHAP/model)
      - Uses NumPy arrays in plots to avoid pandas rename/index issues
      - Shows friendly labels (aliases) for plotting only
      - Enforces model's training feature order if available

    Parameters
    ----------
    shap_values : array | list[array] | shap.Explanation
        Output from your explain step.
    explainer : shap explainer
        The explainer returned by explain_model_shap.
    importance_df : pd.DataFrame | None
        Accepted to match older call sites; not required for plotting.
    X_test : pd.DataFrame
        The same data passed to explanation (or a superset with same columns).
    show_summary, show_bar, show_waterfall : bool
        Toggle plots.
    feature_alias_map : dict[str, str] | None
        Mapping from ORIGINAL feature names -> pretty labels (for display only).
    feature_names : list[str] | None
        If provided, use this exact order (original training names).
    class_index : int | None
        For multiclass SHAP arrays, which class to visualize (default 0).
    clean_labels : bool
        If True, remove parentheses (and their contents), replace slashes with spaces, and squash spaces.
    """
    import numpy as np
    import pandas as pd
    import re
    import shap

    def _clean(s: str) -> str:
        s = re.sub(r"\(.*?\)", "", s)
        s = s.replace("/", " ")
        s = re.sub(r"\s+", " ", s).strip()
        return s

    # 1) Copy input and determine the correct ORIGINAL feature order
    if not isinstance(X_test, pd.DataFrame):
        raise ValueError("X_test must be a pandas DataFrame for plotting.")

    X_df = X_test.copy().reset_index(drop=True)

    # Preferred order: explicit 'feature_names' param (original names)
    if feature_names is not None:
        expect = list(feature_names)
    else:
        # Next preference: model's training order, if available through the explainer
        model_in_explainer = getattr(explainer, "model", None)
        if model_in_explainer is not None and hasattr(model_in_explainer, "feature_names_in_"):
            expect = list(model_in_explainer.feature_names_in_)
        else:
            # Fallback: whatever order is currently in X_df
            expect = list(X_df.columns)

    # Ensure X_df has exactly those columns in that order
    missing = [c for c in expect if c not in X_df.columns]
    if missing:
        raise ValueError(f"Missing expected training columns in X_test for plotting: {missing}")
    X_df = X_df.loc[:, expect]
    ordered_feature_names = expect

    # 2) Build pretty labels WITHOUT renaming data  ✅
    if feature_alias_map:
        display_names = [feature_alias_map.get(c, c) for c in ordered_feature_names]
    else:
        display_names = ordered_feature_names

    if clean_labels:
        display_names = [_clean(s) for s in display_names]

    # 3) Normalize shap_values to a 2D array (n_samples, n_features)
    if isinstance(shap_values, list):
        # Multiclass: pick class_index (default 0)
        k = class_index if class_index is not None else 0
        values = np.asarray(shap_values[k])
    else:
        values = shap_values.values if hasattr(shap_values, "values") else np.asarray(shap_values)

    # 4) Use NumPy for plotting to avoid pandas rename/index paths
    X_np = X_df.to_numpy()

    # Summary (beeswarm)
    if show_summary:
        print("\n🔹 SHAP Summary (Beeswarm) Plot:")
        shap.summary_plot(
            values,
            features=X_np,                 # NumPy, not DataFrame
            feature_names=display_names,   # pretty labels
            show=True,
            max_display=len(display_names)
        )

    # Bar (mean |SHAP|)
    if show_bar:
        print("\n🔹 SHAP Feature Importance (Bar) Plot:")
        shap.summary_plot(
            values,
            features=X_np,
            feature_names=display_names,
            plot_type="bar",
            show=True,
            max_display=len(display_names)
        )

    # Waterfall for the first instance
    if show_waterfall:
        try:
            base = explainer.expected_value
            # If expected_value is vector-like (e.g., multiclass), pick the same class as above
            if isinstance(base, (list, tuple, np.ndarray)):
                idx = class_index if class_index is not None else 0
                base = np.ravel(base)[idx]

            ex0 = shap.Explanation(
                values=values[0],
                base_values=base,
                data=X_np[0],
                feature_names=display_names
            )
            print("\n🔹 SHAP Waterfall Plot for First Instance:")
            shap.plots.waterfall(ex0, show=True)
        except Exception as e:
            print(f"⚠️ Waterfall plot skipped: {e}")

    return True
